# Statistische Analyse

Auch wenn es langweilig klingt - die statistische Analyse der Daten ist extrem wichtig. Aus unserer Erfahrung würden wir vermuten, dass die meisten Analyse- und Machine Learning-Projekte scheitern, weil die Statistik der Grunddaten nicht stimmt.

Diese müssen wir daher unbedingt am Anfang überprüfen. Wir werden versuchen, das so interessant wie möglich zu gestalten und dabei auch verschiedene Visualisierungen verwenden.

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
from IPython.core.pylabtools import figsize
figsize(16, 9)

## Ab hier geht es nun auch wieder mit Colab weiter!

Die Daten des Transport-Flairs stellen wir als SQLIte-Datenbank zur Verfügung, dabei handelt es sich immer noch um eine beträchtliche Menge.

Zunächst bauen wir die Verbindung zur Datenbank auf:

In [ ]:
!test -f technology-transport-short.db || wget https://datanizing.com/data-science-day/technology-transport-short.7z && 7z x technology-transport-short.7z

In [ ]:
sql = sqlite3.connect("technology-transport-short.db")

Die Datenbank selbst ist sehr einfach aufgebaut und besteht nur aus einer einzigen Tabelle (eigentlich könnte man auch mit CSV-Dateien arbeiten, aber dann müssen immer alle Daten im Speicher gehalten werden - das ist oft ungeschickt):

|Feld|Typ|Attribute|
|---|---|---|
|id|text|not null primary key|
|kind|text||
|title|text||
|link_id|text||
|parent_id|text||
|ups|integer||
|downs|integer||
|score|integer||
|author|text||
|num_comments|integer||
|created_utc|timestamp||
|permalink|text||
|url|text||
|text|text||
|level|integer||
|top_parent|text||

Bei User Generated Content oder auch anderen Textdaten ist es häufig sinnvoll, die Analyse in zwei unterschiedlichen Domänen durchzuführen.

Wir betrachten zunächst die strukturierten Daten (die *Metadaten*), um zu überprüfen, ob die statistisch signifikant und valide sind.

Anschließend konzentrieren wir uns auf die unstrukturierten Daten, also die Texte selbst. Dort versuchen wir herauszufinden, ob die für uns relevanten Themen abgedeckt werden.

## Gesamtstatistik

Wir starten mit der Berechnung einiger Größen, die wir später immer wieder benötigen werden. Diese sind noch nicht zeitabhängig, sondern sollen uns nur einen Eindruck von der Größe der Datenmenge verschaffen.

Als erstes interessiert uns die Gesamtanzahl der Posts:

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM posts", sql)

Das ist eine sehr große Anzahl von Posts, darin enthalten sind allerdings sowohl Initial-Posts als auch Kommentare auf Posts. 

Wenn wir nur die Toplevel- oder Initial-Posts betrachten wollen, so ist das leicht möglich, weil bei diesen keine `parent_id` gesetzt ist:

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM posts WHERE parent_id IS NULL", sql)

Auch diese Anzahl ist absolut groß genug, um daraus statistisch signifikante Aussagen ableiten zu können.

Oftmals haben Foren oder UGC-Sites das Problem, dass die Inhalte zwar in großer Menge zur Verfügung stehen, aber nur von wenigen Autoren geschaffen werden:

In [ ]:
pd.read_sql("SELECT COUNT(DISTINCT author) FROM posts", sql)

Auch hier herrscht eine große Vielfalt, was uns in der Analyse sehr hilft. Betrachten wir außerdem noch die Anazhl der Autoren, die die Toplevel-Posts erstellt haben:

In [ ]:
pd.read_sql("SELECT COUNT(DISTINCT author) FROM posts WHERE parent_id IS NULL", sql)

Im Verhältnis zu der Anzahl der Toplevel-Posts sind das ziemlich viele Autoren, so dass sich auch hier ein breites Meinungsbild ergibt.

Schauen wir uns zuletzt noch die Namen der Autoren an, die am meisten geschrieben haben:

In [ ]:
pd.read_sql("SELECT author, COUNT(author) AS count FROM posts \
             WHERE parent_id IS NULL\
             GROUP BY author ORDER BY count DESC LIMIT 20", sql)

Wir können hier erkennen, dass es schon sehr aktive Autoren gibt. Das ist aber ein übliches Verhältnis bei sozialen Netzwerken, das Phänomen nennt sich [Ein-Prozent-Regel](https://de.wikipedia.org/wiki/Ein-Prozent-Regel_(Internet)).

Bis hierher sieht also alles gut aus, die Grundwerte der Datenmenge passen!

## Zeitentwicklung von Posts und Kommentaren

Nach der Business-Fragestellung möchten wir versuchen, Trends abzuleiten und vergangene Trends zu verstehen. Das geht allerdings nur, wenn die Daten hinlänglich aktuell sind. Dewegen analysieren wir zunächst den zeitlichen Verlauf der Posts.

Dazu verdichten wir die Daten gleich nach Monaten, eine genauere Analyse ist für den langen Zeitraum nicht sinnvoll:

In [ ]:
time = pd.read_sql("SELECT STRFTIME('%Y-%m-01', created_utc) AS time, COUNT(*) AS count \
                    FROM posts GROUP BY time", 
                   sql, parse_dates=["time"])

In [ ]:
time.set_index("time").plot(title="Gesamtposts pro Monat")

Die Grafik wirkt etwas *unruhig*, weil sich von Monat zu Monat doch größere Änderungen ergeben. Das können wir glätten, indem wir die Posts auf Quartale verdichten. `pandas` bietet uns dazu leistungsfähige Funktionen:

In [ ]:
time.set_index("time").resample('Q').sum().plot(title="Gesamtposts pro Quartal")

Das sieht deutlich übersichtlicher aus. Es ist außerdem ein positiver Trend zu beobachten, was uns zuversichtlich stimmen sollte, dass wir statistisch valide Daten analysieren.

## Statistik über Autoren

Neben den Posts selbst spielen die Autoren eine große Rolle. Wie haben die sich über die Zeit entwickelt? So wäre es etwa ungünstig, wenn es immer weniger Autoren gibt. Auch hier kann uns die Datenbank viel Rechenarbeit abnehmen:

In [ ]:
time_author = pd.read_sql("SELECT STRFTIME('%Y-%m-01', created_utc) AS time, \
                                  COUNT(DISTINCT author) AS count \
                           FROM posts WHERE parent_id IS NULL GROUP BY time", 
                          sql, parse_dates=["time"])

Der Übersichtlichkeit halber aggregieren wir das gleich wieder für Quartale:

In [ ]:
time_author.set_index("time").resample('Q').sum().plot(title="Autoren pro Quartal")

Das ist sehr interessant! Offenbar hat die Anzahl der Autoren mit der Zeit abgenommen seit dem Maximum im Jahr 2012. Da die Anzahl der Posts gewachsen ist, bedeutet das, dass die durchschnittliche Anzahl von Posts pro Autor auch zugenommen haben muss. Das ist durchaus typisch für eine Experten-Community.

Betrachten wir das Posting-Verhalten der Autoren noch etwas genauer und berechnen dazu eine Tabelle, in der für jeden Autor die Anzahl der Posts steht:

In [ ]:
cpa = pd.read_sql("SELECT author, COUNT(*) AS c FROM posts GROUP BY author", sql)

Die Posts der gelöschten Accounts interessieren uns nicht (weil sich dahinter vermutlich Einzelposts vieler unterschiedlicher Autoren verbergen). Auch den `AutoModerator` lassen wir für die Analyse weg und erzeugen damit einen kleineren `DataFrame`:

In [ ]:
cpa = cpa[~cpa["author"].isin(["[deleted]", "AutoModerator"])]

Mithilfe der `describe`-Funktion können wir uns statistische Informationen zu der Anzahl der Posts pro Autor ausgeben lassen:

In [ ]:
cpa.describe()

Tatsächlich hat ein einzelner Autor 3.100 Posts verfasst. Die große Mehrzahl der Autoren (mehr als die Hälfte) hat hingegen nur einmal gepostet.

Das können wir uns gut in einem Histogramm darstellen lassen. Aufgrund der sehr ungleichen Verteilung wählen wir eine logarithmische Darstellung:

In [ ]:
cpa.plot.hist(bins=80, logy=True)

Auch wenn die Verteilung etwas merkwürdig aussieht, passt das gut! Es gibt nur vier Autoren, die mehr als 1.000 Posts geschrieben haben. Das spricht für eine gut *balancierte* Community.

## Korrelationsanalyse

In Reddit können sog. *Scores* vergeben werden. Dahinter verbirgt sich die Differenz von Up- und Downvotes (die einzeln nicht mehr sichtbar sind). Wir können die Hypothese aufstellen, dass Toplevel-Posts mit einem hohen Score auch viele Kommentare auf sich ziehen.

Dazu lassen wir wieder die Datenbank den Großteil der Berechnung erledigen. Um statistisch signifikante Informationen zu produzieren, betrachten wir nur Posts mit mehr als zehn Kommentaren:

In [ ]:
sc_top = pd.read_sql("SELECT p.id, AVG(p.score) AS score, COUNT(*) AS count\
                       FROM posts p, posts c\
                       WHERE p.parent_id IS NULL AnD c.parent_id=p.id \
                       GROUP BY p.id\
                       HAVING count>10", sql)

Diese können wir nun in einem sog. Scatterplot darstellen:

In [ ]:
sc_top.plot.scatter(x="score", y="count")

Hier scheint es tatsächlich eine Korrelation zu geben. Eine bessere Darstellung können wir mithilfe von `seaborn` erreichen:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 16))
sns.jointplot(x="score", y="count", data=sc_top, kind="reg")

das sieht nach einem deutlichen Zusammenhang aus. Wenn wir das quantifizieren wollen, führen wir eine Regressionsanalyse durch:

In [ ]:
import scipy.stats
r = scipy.stats.linregress(sc_top["score"], sc_top["count"])
r

Der *Pearson R*-Wert ist 1, wenn Werte total korreliert sind, -1 bei einer Antikorrelation und 0 bei unkorrelierten Werten. Auch hier kann man die Korrelation gut erkennen. `p` ist das sog. Signifikanzniveau und hier sehr klein, was für die Güte der Analyse spricht.

Nachdem der Score in etwa den Likes in anderen sozialen Netzwerken entspricht, haben wir hier den bekannten Zusammenhang zwischen Likes und Comments nachgewiesen.

## Inhaltliche Analyse

Wir kennen nun die Statistik der Metadaten und einige Korrelationen, die uns zuversichtlich stimmen, dass der Transport-Flair des Technology-Subreddit gut geeignet für unsere Analyse ist.

Allerdings müssen wir noch die inhaltliche Seite überprüfen. So wäre es z.B. möglich, dass das Reddit voller Spam-Nachrichten ist oder die Diskussion trotz des Namens in eine völlig andere Richtung gehen. Dazu müssen wir die Texte analysieren.

Wir laden zunächst die Title und Texte aus der Datenbank ein: 

In [ ]:
posts = pd.read_sql("SELECT title, text, title||' '||text AS fulltext FROM posts", sql)

Um die einzelnen Wörter zu zählen, ist der `Counter` aus dem `collections`-Paket von Python optimal:

In [ ]:
from collections import Counter

Wir betrachten zunächst die Titel und müssen diese nun in Wörter zerlegen. [Tokenisierung](https://de.wikipedia.org/wiki/Tokenisierung) ist ein nicht-triviales Problem, das man normalerweise mit spezieller Software wie etwas [spaCy](https://spacy.io) lösen sollte. Das sparen wir uns allerdings hier und nutzen einen einfache `regex`-Tokenizer, weil wir sonst sehr lange auf die Ergebnisse waren müssten.

In [ ]:
import regex as re
title_counter = Counter([w.lower() for t in posts["title"] for w in re.findall(r'[\w-]*\p{L}[\w-]*', t)])

Der `title_counter` verfügt über eine `most_common`-Funktion, mit der wir uns die häufigsten Wörter ausgeben lassen könnten. Stattdessen nutzen wir Wordclouds, die uns eine intuitive Visualisierung bieten:

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
wc.generate_from_frequencies(title_counter)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')

Leider kann man außer sehr allgemeinen Wörtern nicht viel erkennen. Wir müssen die Ergebnisse filtern und die sog. *Stoppworte* eliminieren. Zum Glück gibt es dazu fertige Listen, die wir hier noch etwas ergänzen: 

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
for w in "removed deleted post message account moderators http https www youtube com \
          watch gt look looks feel test know think go going submission link apologize \
          inconvenience don want automatically based buy compose good image karma like \
          lot need people self shit sound sounds spam submitting subreddit things \
          video way years time days doesn en fuck money org read reddit review \
          right said says subreddit subreddits sure thank try use videos wiki \
          wikipedia work ll thing point ve actually wait hello new amp better \
          isn yeah probably pretty yes didn pay long posts commenting portion \
          contribute questions unfortunately allowed submissions gifs pics sidebar".split(" "):
    stopwords.add(w)

Wir nutzen diese Liste und lassen einbuchstabige Wörter auch gleich weg:

In [ ]:
title_counter = Counter([w for t in posts["title"].str.lower()
                            for w in re.findall(r'[\w-]*\p{L}[\w-]*', t)
                               if (w not in stopwords) and (len(w) > 1)
                        ])

Die Wordcloud kann wieder genauso erzeugt werden:

In [ ]:
wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
wc.generate_from_frequencies(title_counter)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')

Das sieht schon sehr gut aus und passt genau zu unserem Thema. Wunderbar, das bedeutet, dass wir die richtige Datenmenge ausgewählt haben und auch unsere Klassifikation gut funktioniert hat.

Analysieren wir zum Vergleich noch die vollständigen Text:

In [ ]:
text_counter = Counter([w.lower() for t in posts["fulltext"].str.lower() 
                            for w in re.findall(r'[\w-]*\p{L}[\w-]*', t)
                               if (w not in stopwords) and (len(w) > 1)
                        ])

In [ ]:
wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
wc.generate_from_frequencies(text_counter)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')

Auch das passt prima!

## Topic Modelle

Bisher haben wir die inhaltlichen Aspekte der Posts nur durch Zählen der Wörter berücksichtigt. Allerdings interessieren uns auch Nischen-Themen, die mir mithilfe sog. [Topic Modelle](https://en.wikipedia.org/wiki/Topic_model) aufdecken können.

Hierbei handelt es sich um ein unüberwachtes Machine Learning-Verfahren zur Aufdeckung der latenten Struktur großer Datenmengen.

Am häufigsten wird für Topic Models die sog. [LDA-Methode](https://de.wikipedia.org/wiki/Latent_Dirichlet_Allocation) eingesetzt, die mit stochastischem Sampling funktioniert. Da die Berechnung sehr lange benötigt und es sich in vielen Projekten gezeigt hat, dass die Ergebnisse des NMF-Algorithmus oft (mindestens) genauso gut sind, nutzen wir diesen.

Dafür werden die Texte im ersten Schritt mit TD/IDF vektorisiert:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_text_vectorizer = TfidfVectorizer(stop_words=stopwords, min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(posts['fulltext'])

Nun können wir das Topic Model instanziieren und berechnen lassen. Bei (fast) allen Topic Models müssen wir die Anzahl der Topics vorgeben. Es gibt bestimmte Metriken wie Perplexität oder Kohärenz, mit denen sich die Güte des Modells bestimmen lässt. In unserem Fall arbeiten wir einfach mit 10 Topics:

In [ ]:
from sklearn.decomposition import NMF
nmf_text_model = NMF(n_components=10, random_state=42)
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)

Die Berechnung dauert normalerweise keine Minute, jetzt können wir die Daten visualisieren. Dafür nutzen wir eine kleine Hilfsfunktion, die über die Topics iteriert und Wordclouds als Ergebnisse darstellt:

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def wordcloud_topic_model_summary(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        freq = {}
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            freq[feature_names[i].replace(" ", "_")] = topic[i]
        wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
        wc.generate_from_frequencies(freq)
        plt.figure(figsize=(12,12))
        plt.imshow(wc, interpolation='bilinear')

Wir können uns nun die Wordclouds für die 10 Topics aus dem Topic Model ausgeben lassen:

In [ ]:
wordcloud_topic_model_summary(nmf_text_model, tfidf_text_vectorizer.get_feature_names(), 40)

Plötzlich können wir auch Nischenthemen erkennen, die uns vorher verborgen waren. Das ist sehr praktisch, um Ideen für Trends zu identifizieren. Hiermit können wir außerdem erkennen, ob bestimmte Wörter möglicherweise noch eliminiert werden müssen (wie z.B. `deleted post`, das sich deswegen auch in den Stopwords findet).

Durch die Geschwindigkeit, mit der ein NMF-Topic Model berechnet werden kann, bietet sich diese Methode auch zur Qualitätssicherung an.